In [1]:
from google.colab import files
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import xgboost as xgb
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

! pip install scikit-optimize  #(https://towardsdatascience.com/hyperparameter-optimization-with-scikit-learn-scikit-opt-and-keras-f13367f3e796)
files.upload()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


     |████████████████████████████████| 102kB 3.8MB/s 


Saving train.csv to train.csv


In [2]:
df=pd.read_csv('train.csv')
df
sites=['site%s' %i for i in range(1,11)]
sites=df[sites]

time=['time%s' %i for i in range(1,11)]
time=df[time]

scale = MinMaxScaler()
normalized_sites=scale.fit_transform(sites)
normalized_sites=pd.DataFrame(normalized_sites)
normalized_sites

,0,1,2,3,4,5,6,7,8,9
0,0.017236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.021370,0.022597,0.092456,0.022597,0.022621,0.092430,0.092454,0.092428,0.036418,0.036466
2,0.355000,0.000913,0.354993,0.355017,0.000865,0.000913,0.354984,0.354976,0.354976,0.354976
3,0.018774,0.018774,0.018775,0.018775,0.018775,0.018774,0.018774,0.018774,0.018774,0.018774
4,0.000505,0.004231,0.004183,0.004255,0.004231,0.004255,0.004183,0.004231,0.004231,0.004255
...,...,...,...,...,...,...,...,...,...,...
253556,0.083486,0.083488,0.003366,0.058344,0.002524,0.058343,0.058343,0.058341,0.058341,0.003534
253557,0.305913,0.305921,0.053224,0.000889,0.053224,0.000529,0.611625,0.053221,0.000529,0.000481
253558,0.063942,0.360658,0.133684,0.133684,0.133684,NaN,NaN,NaN,NaN,NaN
253559,0.019495,0.016226,0.019544,0.000505,0.000914,0.019496,0.019544,0.013678,0.000505,0.013678


In [3]:
# Convert to datetime
time=time.apply(pd.to_datetime,dayfirst=True,errors='coerce')

# Find max and min of each row
time['max']=time.loc[:,'time1':'time10'].max(axis=1)
time['min']=time.loc[:,'time1':'time10'].min(axis=1)

# find the difference
time['sess_total']=time['max']-time['min']

#Convert the difference[dtype: timedelta64]  to seconds
time['sess_total']=(time['sess_total'] / np.timedelta64(1, 's')).astype(int)

# Add Other features
time['hour']=time.time1.dt.hour
time['minute']=time.time1.dt.minute
time['second']=time.time1.dt.second
time['month']=time.time1.dt.month
time['day']=time.time1.dt.day
time['month']=time.time1.dt.month
time['year']=time.time1.dt.year
time['dow']=time.time1.dt.dayofweek
time['ifweekend']=time['dow'].apply(lambda x: 1 if x in[5,6] else 0)
time['PoD']=time['hour'].apply(lambda x: 1 if x in range (0,8) else ( 2 if x in range(8,16) else 3))

# Merge the data
df=pd.concat([df[['target']],time.iloc[:,12:],normalized_sites],axis=1)

df=df.fillna(0)

y=df.target
df=df.drop(columns=['target'])

In [4]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

def find_bp(clf,params):
  bs=BayesSearchCV(clf,params,n_iter=32, n_jobs=-1)
  bs.fit(df,y)
  print(bs.best_estimator_)
  return bs.best_estimator_


In [ ]:
# Logistic Regression

# params = {
#           'C' : (np.logspace(-4, 4, 20)),
#          }


# clf = find_bp(LogisticRegression(), params)

clf=LogisticRegression(C=78.47599703514607, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=800, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

clf.fit(df, y)
ypred=clf.predict(df)

score = cross_val_score(estimator=clf, X=df, y=y, cv=3, scoring='roc_auc').mean()
score

0.8205186815821716

In [6]:
# KNN

# params={
#          'leaf_size' : Integer (1,50),
#          'n_neighbors' : Integer(5,10),
#          'p': Real(1,2),
#         }

# clf = find_bp(KNeighborsClassifier(),params)

clf=KNeighborsClassifier(algorithm='auto', leaf_size=50, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=61, p=1.0,
                     weights='uniform')

clf.fit(df, y)
ypred=clf.predict(df)

score = cross_val_score(estimator=clf, X=df, y=y, cv=5, scoring='roc_auc').mean()
score

0.8649020037040295

In [ ]:
# LightGBM

import lightgbm as lgb

# params = {
#           'num_leaves' : Integer (10,200),
#           'min_data_in_leaf': Integer(20, 80),
#           'max_depth' :  Integer (5,50),
#           'bagging_fraction':Integer (0.8, 1),
#           'max_bin':Integer (20,90),
#           }

# clf=find_bp(lgb.LGBMClassifier(),params)  

clf=lgb.LGBMClassifier()

clf.fit(df, y)
ypred=clf.predict(df)

score = cross_val_score(estimator=clf, X=df, y=y, cv=5, scoring='roc_auc').mean()
score

0.9841611264805463